This code enables to record reports, nosie them with radio sound, denoise them, use whisper to convert them into text and then calls the mistral API to edit regular reports. 

In [1]:
import os 
import matplotlib.pyplot as plt
from scipy.io import wavfile
import noisereduce as nr
import IPython.display as ipd
import numpy as np
import scipy
import sounddevice as sd
from scipy.io.wavfile import write
import os
import whisper
from mistralai import Mistral
import pydub

c:\Users\grego\miniconda3\envs\map588\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\grego\miniconda3\envs\map588\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
noise_file='radio_noise.wav'

In [3]:
noise=scipy.io.wavfile.read('data/'+noise_file)
frqcy_noise=noise[0]
noise=noise[1].T[0]
noise=noise/np.max(np.abs(noise))
noise=np.concatenate([noise]*20)


In [4]:
def denoise(signal, frqcy): # a bit hardcore, would be a good idea to find another way to do the denoising
    nperseg=int(frqcy)
    _, _,  Sxx = scipy.signal.spectrogram(signal, frqcy, nperseg=nperseg, noverlap=0)
    sum=Sxx.sum( axis=0)
    sum= sum/np.max(sum)
    sum=np.concatenate([sum, np.zeros(1)])
    voice_only_speak=np.array([signal[i] for i in range(len(signal)) if sum[int(i/nperseg)]>0.1 ])
    noise_only=np.array([signal[i] for i in range(len(signal)) if sum[int(i/nperseg)]<0.1 ])
    return nr.reduce_noise(y=voice_only_speak, sr=frqcy, y_noise=noise_only)

In [5]:
def noise_signal(signal, noise, add_gaussian=False): 
    gaussian = np.random.normal(0, 0.5, len(signal))
    gaussian = gaussian/np.max(np.abs(gaussian))
    return signal + 2*noise[0:len(signal)] + (gaussian if add_gaussian else 0)*0.2

In [ ]:
with open('./clef_mistral.txt', 'r') as file:
    clef_mistral=file.read()


'jk4INoOWgdcEbFjhu8AjUkNcJfsERj2y'

In [7]:
# set recording = true if you want to record your own voice
recording = False
if recording : 

    duration = 12  # secondes
    fs = 16000  # Taux d'échantillonnage (16 kHz pour la reconnaissance vocale)

    # Enregistrement audio en mono (1 canal)
    print("Recording...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)  # channels=1 pour mono
    sd.wait()  # Attendre la fin de l'enregistrement
    print("Recording finished")

    # Sauvegarder en fichier WAV avec le bon format (PCM 16-bit)
    write('cr5.wav', fs,recording)  # Conversion en 16 bits


In [8]:
for file in os.listdir('./cr_radios_audio/'): 
    print(file)
    # open the wav and noise it 
    rate, data = wavfile.read('./cr_radios_audio/'+file)
    noised_data = noise_signal(data, noise)
    noised_data=noised_data.reshape(noised_data.shape[0],1)
    # noised data as numpy array, dtype = float32
    noised_data = np.array(noised_data, dtype=np.float32)

    # save the noise data as a new wav file
    # with open('./cr_radios_audio_noised/'+file, 'wb') as f:
    scipy.io.wavfile.write('./cr_radios_audio_noised/' + file, rate, noised_data)



cr1.wav
cr2.wav
cr3.wav
cr4.wav
cr5.wav


In [9]:
# denoising : 
for file in os.listdir('./cr_radios_audio_noised/'): 
    print(file)
    # open the wav and noise it 
    rate, data = wavfile.read('./cr_radios_audio_noised/'+file)
    denoised_data = denoise(data, rate)
    denoised_data=denoised_data.reshape(denoised_data.shape[0],1)
    # noised data as numpy array, dtype = float32
    denoised_data = np.array(denoised_data, dtype=np.float32)
    scipy.io.wavfile.write('./cr_radios_audio_denoised/' + file, rate, denoised_data)

cr1.wav
cr2.wav
cr3.wav
cr4.wav
cr5.wav


In [10]:

# Charger le modèle Whisper
model = whisper.load_model("base")

os.environ["FFMPEG_BINARY"] = r"C:\\Users\\grego\\Downloads\\ffmpeg-7.1\\bin\\ffmpeg.exe"  # Remplacez le chemin par votre chemin correct

audios = os.listdir('./cr_radios_audio_denoised/')
crs=[]
# Charger le fichier audio
for audio in audios :
    # Vérifier si le chemin est correct
    try:
        result = model.transcribe('./cr_radios_audio_noised/'+audio, language="en")  # "fr" pour le français
        print("Texte transcrit :")
        print(result["text"])
        crs.append(result['text'])
        with open('./cr_radios_text/'+audio.split('.')[0]+'.txt', 'w') as f:
            f.write(result["text"])
    except Exception as e:
        print(f"Erreur lors de la transcription : {e}")




c:\Users\grego\Miniconda3\envs\map588\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Texte transcrit :
 is Alpha Squad crossing Route 6 heading north, advancing toward the Mule located further at the spot at fresh vehicle tracks heading toward the top.
Texte transcrit :
 Here is Bravo Squad. We are positioned at the mill, visual on an approaching track. Six max individuals disembarked and starts unloading the hill. They are pulling out packages and heading to those slight contraband. We are moving to Shadow Bay, to monitor their roads. Over.
Texte transcrit :
 Here is Alpha Squad. We have got eyes on six individuals descending their slope to the world or position. They are carrying a loud fags taking covered intercept them over and they are carrying a loud fags taking covered intercept them over
Texte transcrit :
 Here is Bravo Squad. We lost visual on the smugglers returning to the bay scan for rigging over.
Texte transcrit :
 Here is Alpha Squad. Suspects are detained. They were transporting architects, requesting reinforcements for transport to holding the shield.


In [ ]:
messages = [{
    "role": "system",
    "content": "You are a helpful assistant for military synthesis. You receive reports from the radio emitted from different teams on the ground. Each team has partial point of view but you deliver updates of the evolution of the global situation and make links between the different reports. You will answer at each message with a summary of the global situation for the officer."
    }]
client = Mistral(api_key=clef_mistral)
model = "mistral-large-latest"
count=len(crs)
for i in range(len(crs)):     
    # user_input = input("User: ")
    user_input=crs[i]
    messages.append({"role": "user", "content": user_input})
    response = client.chat.complete(
        model=model,  # ou un autre modèle disponible
        messages=messages,
    )
    resp=(response.choices[0].message.content)
    print(resp)
    messages.append({"role": "assistant", "content": resp})
    print()
    print('-------------------')
    print()

messages.append({"role": "user", "content": 'make a global report'})
response = client.chat.complete(
    model=model,  
    messages=messages,
)
resp=(response.choices[0].message.content)

**Global Situation Update:**

- **Alpha Squad** is currently crossing Route 6, moving north. Their objective is to intercept the Mule (enemy vehicle) which has been spotted ahead, following fresh vehicle tracks leading towards the hilltop.

- No other squads or enemy movements reported at this time.

- No engagements or contacts reported by Alpha Squad.

- Terrain elevation increases ahead (hilltop), which may provide the Mule with a strategic high ground position.

- No updates on the Mule's crew, cargo, or destination.

- Recommend caution as Alpha Squad advances; suggest coordination with other squads to cut off potential escape routes or reinforcements for the Mule.

- Awaiting further updates from other teams and Alpha Squad's progress.

-------------------

**Global Situation Update:**

- **Alpha Squad** is advancing north across Route 6, pursuing the Mule toward the hilltop.

- **Bravo Squad** is positioned at the mill, has spotted an enemy vehicle approaching with approximately